In [1]:
import statfin
import pandas as pd
import numpy as np
from scipy.optimize import Bounds, LinearConstraint, SR1, minimize
from matplotlib import pyplot as plt

In [4]:
from local import *
DATA = Data("HVT_TULOT_70")

print(DATA.row(1).sum)
print(DATA.row(1).N)
print(DATA.row(1).p)

658229778
349477
[0.1  0.2  0.25 0.3  0.4  0.5  0.6  0.7  0.75 0.8  0.9 ]


In [ ]:
class Domain:
    def __init__(self, xmax = 1e6, N = 1_000):
        k = np.log(xmax) / (N - 2)
        self.N = N
        self.x = np.hstack([[0], np.exp(k * np.arange(0, N - 1, 1))])
        self.dx = np.diff(self.x)
        self.dx2 = self.dx ** 2

    def index(self, x):
        return round(x)

In [ ]:
class Problem:
    def __init__(self, domain, specs):
        self.domain = domain
        self.specs = specs
        self.bounds = Bounds(0, np.inf)
        self.constraints = []
        self.constraints.append(self.sum_constraint())

    def sum_constraint(self):
        return LinearConstraint(self.domain.x, self.specs.sum, self.specs.sum)

    def cost(self, n):
        return np.sum(self.r(n) - self.domain.dx)

    def jac(self, n):
        dn = np.diff(n)
        return dn / self.r(n)

    def r(self, n):
        dn = np.diff(n)
        dn2 = dn ** 2
        dx2 = self.domain.dx2
        return np.sqrt(dx2 + dn2)

    def minimize(self):
        n0 = self.specs.sum / self.domain.N * np.ones(self.domain.N)
        return minimize(
            self.cost,
            n0,
            #method="trust-constr",
            #jac=self.jac,
            bounds=self.bounds,
            constraints=self.constraints
        )

In [ ]:
p = Problem(Domain(), DATA.row(1))
opt = p.minimize()

In [ ]:
plt.plot(p.domain.x, opt.x)